In [6]:
df = pd.read_csv('data_unclean.csv', sep=';')

In [9]:
df.to_csv('data.csv', index=False)

In [10]:
df.columns

Index(['Exercise Name', 'Type of Activity', 'Type of Equipment', 'Body Part',
       'Type', 'Muscle Groups Activated', 'Instructions'],
      dtype='object')

In [12]:
documents = df.to_dict(orient='records')

## Start from Here ##

In [1]:
import minsearch

In [2]:
import pandas as pd

In [13]:
df = pd.read_csv('data.csv')
df.columns = df.columns.str.lower()

documents = df.to_dict(orient='records')

In [14]:
df.columns

Index(['id', 'document type', 'description', 'issued by', 'used by',
       'mode of transport', 'notes'],
      dtype='object')

In [15]:
df.head()

,id,document type,description,issued by,used by,mode of transport,notes
0,0,Bill of Lading,A legal document issued by a carrier to a ship...,Ocean Carrier or Forwarder,Shipper and Consignee,Sea,"Serves as receipt, contract, and document of t..."
1,1,Air Waybill,Non-negotiable document that covers goods tran...,Airline or Air Forwarder,Shipper and Consignee,Air,Acts as a contract and proof of shipment
2,2,House Bill of Lading,Issued by a freight forwarder to a shipper as ...,Freight Forwarder,Shipper,Sea,Represents the forwarder’s contract of carriage
3,3,Master Bill of Lading,Issued by the main carrier to the freight forw...,Shipping Line,Freight Forwarder,Sea,Shows details of multiple shipments under one ...
4,4,Commercial Invoice,Lists goods sold and their value for customs a...,Exporter,Importer,All,Used for customs valuation and payment


In [71]:
documents[15]

{'id': 15,
 'document type': 'Shipper’s Letter of Instruction',
 'description': 'Provides specific instructions to freight forwarder',
 'issued by': 'Shipper',
 'used by': 'Forwarder',
 'mode of transport': 'All',
 'notes': 'Guides document preparation'}

In [69]:
index = minsearch.Index(
    text_fields=['document type', 'description', 'issued by', 'used by',
       'mode of transport', 'notes'],
    keyword_fields=['id']
)

In [70]:
index.fit(documents)

In [82]:
query = "What is HBL?"

In [83]:
index.search(query, num_results=3)

[]

## Create Rag Logic

In [22]:
import os
from groq import Groq

In [23]:
client = Groq(api_key=os.environ.get("GROQ_API_KEY")) # Initialize the Groq client

In [34]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
       # {"role": "system", "content": "You are a helpful assistant that provides exercise recommendations."},
        {"role": "user", "content": query}
    ],
    max_tokens=200,
)

response = response.choices[0].message.content

In [35]:
print(response)

I'm an artificial intelligence language model, so I don't have real-time access to current weather conditions. However, I can suggest some ways for you to find out the weather for your location.

1. **Check online weather websites**: You can visit websites like AccuWeather, Weather.com, or the National Weather Service (NWS) to get the current weather conditions and forecast for your area.
2. **Use a mobile app**: Download a weather app like Dark Sky, Weather Underground, or The Weather Channel to get real-time weather updates on your smartphone.
3. **Ask a voice assistant**: If you have a smart speaker or virtual assistant like Alexa, Google Assistant, or Siri, you can ask them to tell you the current weather conditions.

Please let me know if there's anything else I can help you with!


In [84]:
def search(query):
    boost = {
        'document type': 3.0,
        'description': 2.5,
        'issued by': 1.2,
        'used by': 1.0,
        'mode of transport': 0.8,
        'notes': 0.5
    }

    results = index.search(
        query=query,
        filter_dict={},  # optional filters later (e.g., mode='Sea')
        boost_dict=boost,
        num_results=5
    )

    return results


In [85]:
prompt_template = """
You're a freight forwarding specialist. Answer the QUESTION based on the provided shipping documents database.
Use only the facts from the KNOWLEDGE BASE when answering the QUESTION.

QUESTION: {question}

KNOWLEDGE BASE:
{context}
""".strip()

entry_template = """
document type: {document type}
description: {description}
issued by: {issued by}
used by: {used by}
mode of transport: {mode of transport}
notes: {notes}
""".strip()


def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [95]:
print(build_prompt(question, search(question)))

You're a freight forwarding specialist. Answer the QUESTION based on the provided shipping documents database.
Use only the facts from the KNOWLEDGE BASE when answering the QUESTION.

QUESTION: what is House BL?

KNOWLEDGE BASE:
document type: House Bill of Lading
description: Issued by a freight forwarder to a shipper as part of consolidated cargo
issued by: Freight Forwarder
used by: Shipper
mode of transport: Sea
notes: Represents the forwarder’s contract of carriage


In [86]:
def llm(prompt,model="llama-3.3-70b-versatile"):
    response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "user", "content": prompt}
    ],
    max_tokens=500,
)
    
    return response.choices[0].message.content

In [87]:
def rag(query, model='llama-3.3-70b-versatile'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [93]:
question = 'what is House BL?'
answer = rag(question)
print(answer)

House BL refers to a House Bill of Lading, which is a document issued by a freight forwarder to a shipper as part of consolidated cargo, representing the forwarder's contract of carriage for sea transport.
